In [1]:
# Leer ficheros de datos
import codecs
import pandas as pd
import pyarrow.parquet as pq
# Extraer información
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
# Auxiliares
import numpy as np
import logging
from typing import Union
import pyarrow as pa
import seaborn as sns
import matplotlib.pyplot as plt

### Cargamos el parquet

In [2]:
df = pd.read_parquet('Datasets/PHA/PHA_Data.parquet', engine='pyarrow')

### Convertimos cualquier tipo de registro faltante a NaN

In [3]:
def clean_data(value): 
    # Si se trata de un valor en blanco sustituir a NaN 
    try: 
        if value in [None, 'nan']: 
            return np.nan 
         
        return value 
 
    except Exception as e: 
        return value 
     
# Iterar sobre todas las columnas y comprobar sus valores 
columns = len(df.columns) 
 
for column in df.columns: 
    # Utilizar compresion de listas y aplicar el filtrado de valores 
    values = [clean_data(value) for value in df[column]] 
    df[column] = values 
    columns -= 1 
    print(f'[+] REMAINING_COLUMNS\t{columns}\t', end='\r') 
     
df

,c1,c6,c7,c8,c10,c132,c134,c136,c138,c141,...,c150,c154,c156,c158,c161,c163,c183,c191,c229,c230
0,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,NaN,NaN
2,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,NaN,NaN
3,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,NaN,NaN
4,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,POWERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214882,I,2022,2,6,0930,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19470720.0,NaN
214883,I,2022,2,6,1710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19700731.0,NaN
214884,I,2022,1,31,0832,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19810601.0,19980409.0
214885,I,2022,1,7,0407,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,19520606.0,NaN


### Cargamos cabecera  y convertimos a dataframe

In [4]:
url_legend = 'https://av-info.faa.gov/data/AID/Afilelayout.txt'
# Leemos el contenido de la leyenda
with urlopen(url_legend) as content:
    soup = BeautifulSoup(content, "html.parser")
    soup_lines = str(soup).split('\r\n')

# Transformamos la respuesta en un diccionario con el nombre de la columna y la descripción
legend_df = {"Column_name": [], "Description": []}
# Recorrer las lineas con datos de lineas. Se salta las dos primeras y las tres ultimas
# filas por no tener datos relevantes de la leyenda
for line in soup_lines[2:-3]:
    # Se extrae los 5 primeros caracteres para conocer el nombre de la columna
    legend_df["Column_name"].append(line[0:5].strip())
    # Se extrae la descripcion, esta comienza en la posicion 53
    legend_df["Description"].append(line[53:].strip())
    
# Convertir el diccionario en pandas.Dataframe
legend_df = pd.DataFrame.from_dict(data=legend_df)
legend_df

,Column_name,Description
0,c5,Unique control number used to relate to AID_MA...
1,c1,Type of Event
2,c2,FAR part number
3,c3,Form on which the latest data was received.
4,c4,Agency conducting investigation.
...,...,...
174,c163,2nd Additional cause factor text
175,c183,Supporting Factor Text ...
176,c191,Supporting cause factor B text. O...
177,c229,Date of Birth of PIC


### Enumeramos las cabecera con la descripción para poder verlas mejor y seleccionamos las columnas que podrían ser relevantes

In [5]:
used_columns = ['c1', 'c6', 'c7', 'c8', 'c10', 'c132', 'c134', 'c136', 'c138', 'c141', 'c160', 'c162', 'c23', 'c24', 'c11', 'c14', 'c20', 'c21', 'c102', 'c104', 'c106', 'c108', 'c110', 'c114', 'c115', 'c117', 'c118', 'c61', 'c62', 'c65', 'c67', 'c68', 'c41', 'c45', 'c47', 'c49', 'c50', 'c52', 'c53', 'c54', 'c55', 'c56', 'c59', 'c119', 'c120', 'c121', 'c122', 'c126', 'c127', 'c128', 'c130', 'c78', 'c80', 'c82', 'c84', 'c86', 'c88', 'c90', 'c92', 'c94', 'c96', 'c44', 'c46', 'c48', 'c51', 'c229', 'c230']
#used_columns = [f'c{idx}' for idx in used_columns]

for col in used_columns:
    description = legend_df[legend_df["Column_name"] == col]["Description"].values[0]
    print(f'[+]  Descripción columnas\t{col}\t{description}')
   

[+]  Descripción columnas	c1	Type of Event
[+]  Descripción columnas	c6	Year the accident/incident happened.
[+]  Descripción columnas	c7	Month the accident/incident happened.
[+]  Descripción columnas	c8	Day the accident/incident happened.
[+]  Descripción columnas	c10	Local time of the accident/incident.
[+]  Descripción columnas	c132	First remedial action area code
[+]  Descripción columnas	c134	First remedial action taken code
[+]  Descripción columnas	c136	Second remedial action area code
[+]  Descripción columnas	c138	Second remedial action taken code
[+]  Descripción columnas	c141	Method of notification
[+]  Descripción columnas	c160	Additional cause factor code
[+]  Descripción columnas	c162	2nd Additional cause factor code
[+]  Descripción columnas	c23	Make of the aircraft
[+]  Descripción columnas	c24	Model of the aircraft
[+]  Descripción columnas	c11	Region of the accident/incident location.
[+]  Descripción columnas	c14	City of the accident/incident location.
[+]  Descripc

### Creamos dataframe a partir del primario usando solamente las columnas de la lista anteriormente  generada

In [6]:
data= df.loc[:, used_columns]
data

,c1,c6,c7,c8,c10,c132,c134,c136,c138,c141,...,c90,c92,c94,c96,c44,c46,c48,c51,c229,c230
0,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,A,NaN,KX,IB,ASMEL,INSTRUMENT,FLT INST QUALIFIED I,PROFESSIONAL PILOT,NaN,NaN
1,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,JJ,DD,ASEL,INSTRUMENT,QUAL IN OPER NOT FLI,STUDENT,NaN,NaN
2,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,JA,DD,ASEL,RATINGS SHOWN BY 1ST,QUAL IN OPER NOT FLI,OTHER,NaN,NaN
3,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,AA,HB,ASEL,RATINGS SHOWN BY 1ST,QUAL IN OPER NOT FLI,ENGINEER,NaN,NaN
4,A,1975,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,AA,IA,ASEL,RATINGS SHOWN BY 1ST,QUAL IN OPER NOT FLI,OTHER,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214882,I,2022,2,6,0930,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,CR,NaN,NaN,NaN,NaN,19470720.0,NaN
214883,I,2022,2,6,1710,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,TC,NaN,NaN,NaN,NaN,19700731.0,NaN
214884,I,2022,1,31,0832,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,TD,NaN,NaN,NaN,NaN,19810601.0,19980409.0
214885,I,2022,1,7,0407,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,LT,NaN,NaN,NaN,NaN,19520606.0,NaN


### Realizamos un conteo de los datos faltantes NaN de las columnas que pueden ser potenciales

In [7]:
# Contar el número de valores NaN en cada columna
print(data.isna().sum())

c1           8
c6           0
c7          33
c8          33
c10      21286
         ...  
c46      51213
c48      58745
c51      49902
c229    202962
c230    213480
Length: 67, dtype: int64


### Procedemos a borrar las columnas con demasiados datos faltantes (más de 100.000 unidades) y las filas que tengan más de 3 valores faltantes

In [8]:
#Borramos columnas que tengan más de 100.000 datos faltantes  
data = data.dropna(thresh=len(data) - 100000, axis=1)
df = df.dropna(thresh=3)

### Volvemos a contar despues del borrado para ver que columnas son aprovechables

In [9]:
# Contar el número de valores NaN en cada columna
print(data.isna().sum())



c1          8
c6          0
c7         33
c8         33
c10     21286
c132    59855
c134    56536
c141    78874
c23      1967
c24      1839
c11       508
c14      1847
c102    41436
c104    41518
c106    42958
c108    43218
c110    46444
c61        33
c62      5967
c65      5967
c67      5967
c68      5967
c41     13801
c45     57794
c47     51213
c49     58745
c50     48583
c52     49902
c53     36021
c54     91362
c55     40299
c56     37095
c59     58919
c119     5427
c130    64166
c78     39275
c94     44033
c96      7062
c44     57794
c46     51213
c48     58745
c51     49902
dtype: int64


In [10]:
columnas = list(data.columns)
#unused_columns = [5, 3, 2, 4, 9, 75, 140, 139, 203, 204, 214, 790, 26, 37, 15, 16, 17,18, 19, 143, 205, 206, 207, 208, 210, 43, 129, 124, 125, 77]
#unused_columns = [f'c{idx}' for idx in unused_columns]

for col in columnas:
    description = legend_df[legend_df["Column_name"] == col]["Description"].values[0]
    print(f'[+] Descripción columnas\t{col}\t{description}')

[+] Descripción columnas	c1	Type of Event
[+] Descripción columnas	c6	Year the accident/incident happened.
[+] Descripción columnas	c7	Month the accident/incident happened.
[+] Descripción columnas	c8	Day the accident/incident happened.
[+] Descripción columnas	c10	Local time of the accident/incident.
[+] Descripción columnas	c132	First remedial action area code
[+] Descripción columnas	c134	First remedial action taken code
[+] Descripción columnas	c141	Method of notification
[+] Descripción columnas	c23	Make of the aircraft
[+] Descripción columnas	c24	Model of the aircraft
[+] Descripción columnas	c11	Region of the accident/incident location.
[+] Descripción columnas	c14	City of the accident/incident location.
[+] Descripción columnas	c102	Primary type of flying code
[+] Descripción columnas	c104	Secondary type of flying code
[+] Descripción columnas	c106	Primary flying condition code
[+] Descripción columnas	c108	Secondary flying condition code
[+] Descripción columnas	c110	Light co

### Imputación